<p style='
  color: #3b4045; 
  text-align: center;
  font-weight: bold;
  font-family: -apple-system,BlinkMacSystemFont, "Segoe UI Adjusted","Segoe UI","Liberation Sans",sans-serif;     font-size: 2.07692308rem; '> 
    Vascular Graph Extraction
</p>


In [1]:
%load_ext autoreload
%autoreload 2

from copy import deepcopy

import cv2
import numpy as np
from ipywidgets import GridspecLayout
from jppype import View2D, imshow, sync_views, vscode_theme

from fundus_vessels_toolkit.pipelines import FundusVesselSegToGraph
from fundus_vessels_toolkit.segment import segment_vessels
from fundus_vessels_toolkit.utils.image import load_img

vscode_theme()

HTML(value="<style>\n        .cell-output-ipywidget-background {\n                background: transparent !imp…

In [2]:
# Path to the raw fundus image
RAW_PATH = "/home/gaby/These/Data/Fundus/Vessels/FIVES/test/downsampled-1024/1-images/50_A.jpg"

# Path to a vessel ground truth (for comparison purposes)
VESSELS_PATH = "/home/gaby/These/Data/Fundus/Vessels/FIVES/test/downsampled-1024/2-vessels/50_A.png"

### Load the raw Image and the Vascular Segmentation


Load image and segmentation mask


In [3]:
raw = load_img(RAW_PATH)
true_vessels = load_img(VESSELS_PATH, binarize=True)

Perform segmentation using a pretrained model


In [4]:
vessels = segment_vessels(raw)

---


### Skeletonize and Extract Graph


All the skeletonization and graph extraction pipeline is accessible through `RetinalVesselSeg2Graph` objects.

This object tunes all the parameters of the pipeline according to a single parameter: the maximum vessel diameter.
Apriori, a good approximation is a 80th of the image width (13px for a 1024x1024 px image).


In [5]:
max_vessel_diameter = raw.shape[1] // 80
seg2graph = FundusVesselSegToGraph(max_vessel_diameter)

In [6]:
seg2graph.nodes_merge_distance

NodeMergeDistances(junction=12, termination=12, node=0)

To compute the graph extraction you simply need to call the object with a segmentation map:


In [7]:
skel = seg2graph.skeletonize(vessels)
graph = seg2graph.skel_to_vgraph(skel, vessels)

In [8]:
seg2graph.nodes_merge_distance.junction = 12
seg2graph.nodes_merge_distance.termination = 0
graph2 = seg2graph.simplify_vgraph(graph)

The returned object is a `VascularGraph` which expose all the properties of the extracted graph.

- The list of branches : `vgrapp.branch_list`
- The y and x position of the nodes: `vgraph.nodes_coord()`

This object is also usefull to cast the graph to other representation. For example:

- Node Adjacency Matrix: `vgraph.node_adjacency_matrix()`
- Branch Adjacency Matrix: `vgraph.branch_adjacency_matrix()`


#### Display


Lets compare the graph extracted on the predicted segmentation and on the ground truth segmentation side by side!

_(The prediction is on the left, the ground truth on the right.)_


In [9]:
def create_view(vessels, vgraph, edge_map=True):
    v = imshow(raw)
    v.add_label(vessels, "vessel", "white", options={"opacity": 0.2})
    v.add_label(skel, "skel", "grey", options={"opacity": 0.3})
    v["vessel graph"] = vgraph.jppype_layer(edge_map=edge_map, edge_labels=True)

    return v


grid = GridspecLayout(1, 2, height="650px")
grid[0, 0] = create_view(vessels, graph, True)
grid[0, 1] = create_view(vessels, graph2, True)
sync_views(grid[0, 0], grid[0, 1])

grid

GridspecLayout(children=(View2D(layout=Layout(grid_area='widget001'), linkedTransformGroup='3cc0f9fc925f41a0bf…

---


### Performing only the skeletonization


The method `seg2graph()` actually perform the graph extraction in three steps:

- Skeletonization (morphological reduction of the segmentation to the vessels centerline): `seg2graph.skeletonize(vessel_segmentation_map)`
- Graph extraction (identification of branches and nodes in the vascular skeleton and connection parsing): `seg2graph.skel_to_vgraph(vessel_skeleton_map, vessel_segmentation_map)`.
- Graph simplification (removal of small branches, merging close nodes...): `seg2graph.simplify_vgraph(vgraph)`

Those method can be called individually, for example to get the result of the skeletonization step only.


In [10]:
skeleton = seg2graph.skeletonize(vessels)

You may notice some difference between the skeleton obtained through `seg2graph.skeletonize(vessels)` and by direct call of `seg2graph(vessels)`, because several topological simplification is performed on the latter (merging close nodes, removing small spurs or cycles...).


In [11]:
v = imshow(raw)
v.add_label(vessels, "vessel", "white", options={"opacity": 0.2})
v.add_label(skeleton, "skeleton", {2: "white", 1: "yellow", 3: "red", 4: "purple"})
v

View2D()

<br />
<hr style="width:20%;text-align:center;"/>

Calling `seg2graph.seg2adjacency(vessel_map)` will return the adjacency matrix `adj` which store the graph links between branches (rows) and nodes (columns).

If `return_label=True` this method also returns:

- the label map of branches where the value of each pixel correspond to its branch id **+ 1** (or 0 if the pixel is not part of the skeleton);
- the coordinates of each nodes as a tuple (y, x) where y and x are vectors of length `adj.shape[1]`.


In [12]:
adj, branch_labels, node_yx = seg2graph.seg2adjacency(vessels, return_label=True)

AttributeError: 'FundusVesselSegToGraph' object has no attribute 'seg2adjacency'

In [ ]:
v = imshow(raw)
v.add_label(branch_labels)
v